In [1]:
import pandas as pd
from sklearn import preprocessing
import numpy as np
from sklearn.model_selection import train_test_split
import torch
import torchvision
import torchvision.transforms as transforms

/home/hoangphuc/miniconda3/envs/gene/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
cna_df = pd.read_csv('data/CNA_test_data.csv', index_col=0)

In [3]:
Gtotal_df = pd.read_csv('data/GWFP_total_norm_test_data.csv', index_col=0)

In [4]:
Gshort_df = pd.read_csv('data/GWFP_short_norm_test_data.csv', index_col=0)

In [5]:
Glong_df = pd.read_csv('data/GWFP_long_norm_test_data.csv', index_col=0)

In [6]:
Gratio_df = pd.read_csv('data/GWFP_ratio_test_data.csv', index_col=0)

In [7]:
Gflen_df = pd.read_csv('data/GWFP_flen_range_100_250_test_data.csv', index_col=0)

In [8]:
Gwmd_df = pd.read_csv('data/GWMD_test_data.csv', index_col=0)

In [9]:
tmd_df = pd.read_csv('data/TMD_test_data.csv', index_col=0)

In [10]:
motif_df = pd.read_csv('data/MOTIF_END_test_data.csv', index_col=0)

In [11]:
if len(cna_df.keys()) != 300:
    print("cna")
if len(Gtotal_df.keys()) != 300:
    print("Gtotal")
if len(Gshort_df.keys()) != 300:
    print("Gshort")
if len(Glong_df.keys()) != 300:
    print("Glong")
if len(Gratio_df.keys()) != 300:
    print("Gratio")
if len(Gflen_df.keys()) != 300:
    print("Gflen")
if len(Gwmd_df.keys()) != 300:
    print("Gwmd")
if len(tmd_df.keys()) != 300:
    print("tmd")
if len(motif_df.keys()) != 300:
    print("motif")

In [12]:
df_concat = pd.concat([cna_df, Gtotal_df, Gshort_df, Glong_df, Gratio_df, Gflen_df, Gwmd_df, tmd_df, motif_df], axis=0)

In [13]:
df_concat.head()

,CRC_1,CRC_2,CRC_3,CRC_4,CRC_5,CRC_6,CRC_7,CRC_8,CRC_9,CRC_10,...,Breast_41,Breast_42,Breast_43,Breast_44,Breast_45,Breast_46,Breast_47,Breast_48,Breast_49,Breast_50
1:1000001-2000000,0.896475,0.828850,0.851508,0.927454,0.895133,0.853351,0.883020,0.861244,0.952042,0.949140,...,0.895374,0.932478,0.837540,0.796464,0.932748,0.867180,0.806904,0.849950,0.852024,0.937161
1:2000001-3000000,0.983415,1.001451,1.093219,1.066098,1.077617,1.026893,1.104132,1.002475,1.072812,1.041484,...,0.963836,1.048706,1.070006,1.050492,1.140593,1.035193,1.002272,0.966441,1.050553,1.031910
1:3000001-4000000,1.118625,1.107329,1.137303,1.130093,1.092212,1.126741,1.099327,1.070323,1.129020,1.057470,...,1.104951,1.150284,1.157951,1.029828,1.195422,1.109486,1.081973,1.090978,1.141446,1.114758
1:4000001-5000000,1.113958,1.128191,1.100510,1.082770,1.096592,1.099364,1.116726,1.047043,1.057193,1.074227,...,1.065304,1.080750,1.079786,1.169050,1.101640,1.097720,1.120258,1.159710,1.093195,1.085440
1:5000001-6000000,0.953393,0.935254,0.915066,0.953461,0.921631,0.926536,0.874975,0.988541,0.937228,1.012447,...,0.978225,0.958540,0.917655,0.894931,0.941317,0.974991,0.918791,0.929655,0.939999,0.981326


In [14]:
df_concat.describe()

,CRC_1,CRC_2,CRC_3,CRC_4,CRC_5,CRC_6,CRC_7,CRC_8,CRC_9,CRC_10,...,Breast_41,Breast_42,Breast_43,Breast_44,Breast_45,Breast_46,Breast_47,Breast_48,Breast_49,Breast_50
count,8630.000000,8630.000000,8630.000000,8630.000000,8630.000000,8630.000000,8630.000000,8630.000000,8630.000000,8630.000000,...,8630.000000,8630.000000,8630.000000,8630.000000,8630.000000,8630.000000,8630.000000,8630.000000,8630.000000,8630.000000
mean,26.233177,26.465917,25.778982,25.601706,25.696984,25.593377,24.392744,26.429486,26.277472,26.796369,...,27.132575,26.812305,24.096056,24.731075,25.221298,26.501445,25.697002,25.364958,26.836727,26.983641
std,36.616364,36.863484,35.847756,35.636408,35.877503,35.647806,34.073200,36.663731,36.634156,37.311050,...,37.835779,37.407718,33.799028,34.553995,35.329527,36.991703,35.896014,35.364140,37.359115,37.583846
min,-4.417272,-4.253652,-4.236366,-4.865322,-4.280407,-4.509998,-3.844023,-4.848612,-4.914507,-4.948723,...,-4.908178,-4.844291,-3.751790,-3.064830,-4.212117,-4.801871,-4.180532,-3.515513,-4.676111,-4.903634
25%,0.265869,0.256635,0.298361,0.405172,0.223804,0.300113,0.320041,0.391635,0.402604,0.430808,...,0.394298,0.355435,0.239549,0.281579,0.271376,0.371283,0.353037,0.286459,0.313866,0.401790
50%,1.040370,1.037836,1.049394,1.021436,1.038174,1.040807,1.076876,1.028772,1.021272,1.008297,...,1.021542,1.018306,1.063156,1.082405,1.041675,1.028736,1.053116,1.071066,1.031220,1.019830
75%,74.796722,75.144243,73.795169,73.875324,72.643234,72.241540,67.507509,76.059530,75.530023,77.501896,...,78.315433,77.694258,66.632153,69.077498,69.841235,76.370683,72.877334,70.317470,77.429039,78.027198
max,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,...,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000


In [15]:
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8630 entries, 1:1000001-2000000 to TTTT
Columns: 300 entries, CRC_1 to Breast_50
dtypes: float64(300)
memory usage: 19.8+ MB


# 6 labels

In [16]:
x = df_concat.values #returns a numpy array
X = np.transpose(x)
print(X)
classes = [1]*50 + [0]*50 + [2]*50 + [3]*50 + [4]*50 + [5]*50
Y = np.array(classes)

[[0.89647474 0.98341483 1.11862454 ... 0.00488202 0.00460028 0.00856301]
 [0.82885044 1.00145066 1.10732934 ... 0.00467277 0.00452604 0.00805136]
 [0.85150755 1.09321888 1.13730272 ... 0.00464163 0.00458129 0.00808402]
 ...
 [0.84995006 0.96644121 1.09097766 ... 0.00504033 0.00471977 0.00860973]
 [0.85202402 1.05055339 1.14144582 ... 0.00473525 0.00464    0.00836108]
 [0.93716148 1.03190985 1.11475773 ... 0.00453021 0.00448994 0.00782743]]


In [17]:
# Split into train+val and test
x_trainval, x_test, y_trainval, y_test = train_test_split(X, Y, test_size=0.1, random_state=42)

# Split train into train-val
x_train, x_val, y_train, y_val = train_test_split(x_trainval, y_trainval, test_size=0.1, stratify=y_trainval, random_state=21)
scaler = preprocessing.MinMaxScaler()
x_train = scaler.fit_transform(x_train)
x_val = scaler.transform(x_val)
x_test = scaler.transform(x_test)

In [18]:
with open('data/train_six.npy', 'wb') as f:
    np.save(f, x_train)
with open('data/train_six.npy', 'wb') as f:
    np.save(f, y_train)

with open('data/val_six.npy', 'wb') as f:
    np.save(f, x_val)
with open('data/val_six_target.npy', 'wb') as f:
    np.save(f, y_val)

with open('data/test_six.npy', 'wb') as f:
    np.save(f, x_test)
with open('data/test_six_target.npy', 'wb') as f:
    np.save(f, y_test)

# Control vs Disease => Diseases: 5 labels

In [16]:
x = df_concat.values #returns a numpy array
X = np.transpose(x)
print(X)
classes = [1]*50 + [0]*50 + [1]*50 + [1]*50 + [1]*50 + [1]*50
Y = np.array(classes)

[[0.89647474 0.98341483 1.11862454 ... 0.00488202 0.00460028 0.00856301]
 [0.82885044 1.00145066 1.10732934 ... 0.00467277 0.00452604 0.00805136]
 [0.85150755 1.09321888 1.13730272 ... 0.00464163 0.00458129 0.00808402]
 ...
 [0.84995006 0.96644121 1.09097766 ... 0.00504033 0.00471977 0.00860973]
 [0.85202402 1.05055339 1.14144582 ... 0.00473525 0.00464    0.00836108]
 [0.93716148 1.03190985 1.11475773 ... 0.00453021 0.00448994 0.00782743]]


In [17]:
# Split into train+val and test
x_trainval, x_test, y_trainval, y_test = train_test_split(X, Y, test_size=0.1, random_state=42)

# Split train into train-val
x_train, x_val, y_train, y_val = train_test_split(x_trainval, y_trainval, test_size=0.1, stratify=y_trainval, random_state=21)
scaler = preprocessing.MinMaxScaler()
x_train = scaler.fit_transform(x_train)
x_val = scaler.transform(x_val)
x_test = scaler.transform(x_test)

In [18]:
with open('data/train_bin.npy', 'wb') as f:
    np.save(f, x_train)
with open('data/train_bin_target.npy', 'wb') as f:
    np.save(f, y_train)

with open('data/val_bin.npy', 'wb') as f:
    np.save(f, x_val)
with open('data/val_bin_target.npy', 'wb') as f:
    np.save(f, y_val)

with open('data/test_bin.npy', 'wb') as f:
    np.save(f, x_test)
with open('data/test_bin_target.npy', 'wb') as f:
    np.save(f, y_test)

In [46]:
meta = pd.read_csv("data/metadata.csv")
control_lst = meta['Control_label'].to_list()
diseases_df = df_concat.drop(columns=control_lst)

In [48]:
x = diseases_df.values #returns a numpy array
X = np.transpose(x)
print(X)
classes = [0]*50 + [1]*50 + [2]*50 + [3]*50 + [4]*50
Y = np.array(classes)

[[0.89647474 0.98341483 1.11862454 ... 0.00488202 0.00460028 0.00856301]
 [0.82885044 1.00145066 1.10732934 ... 0.00467277 0.00452604 0.00805136]
 [0.85150755 1.09321888 1.13730272 ... 0.00464163 0.00458129 0.00808402]
 ...
 [0.84995006 0.96644121 1.09097766 ... 0.00504033 0.00471977 0.00860973]
 [0.85202402 1.05055339 1.14144582 ... 0.00473525 0.00464    0.00836108]
 [0.93716148 1.03190985 1.11475773 ... 0.00453021 0.00448994 0.00782743]]


In [49]:
# Split into train+val and test
x_trainval, x_test, y_trainval, y_test = train_test_split(X, Y, test_size=0.1, random_state=42)

# Split train into train-val
x_train, x_val, y_train, y_val = train_test_split(x_trainval, y_trainval, test_size=0.1, stratify=y_trainval, random_state=21)
scaler = preprocessing.MinMaxScaler()
x_train = scaler.fit_transform(x_train)
x_val = scaler.transform(x_val)
x_test = scaler.transform(x_test)

In [50]:
with open('data/train_five.npy', 'wb') as f:
    np.save(f, x_train)
with open('data/train_five_target.npy', 'wb') as f:
    np.save(f, y_train)

with open('data/val_five.npy', 'wb') as f:
    np.save(f, x_val)
with open('data/val_five_target.npy', 'wb') as f:
    np.save(f, y_val)

with open('data/test_five.npy', 'wb') as f:
    np.save(f, x_test)
with open('data/test_five_target.npy', 'wb') as f:
    np.save(f, y_test)

# Check data

In [22]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=42)
print((y_train == 0).sum())
print((y_train == 1).sum())
print((y_train == 2).sum())
print((y_train == 3).sum())
print((y_train == 4).sum())
print((y_train == 5).sum())
x_train, x_test, y_train, y_test = torch.from_numpy(x_train), torch.from_numpy(x_test), torch.from_numpy(y_train).type(torch.LongTensor), torch.from_numpy(y_test).type(torch.LongTensor)

48
43
45
43
40
45


In [24]:
print(x_train[0])
print(x_train.shape)
print(x_test[0])
print(x_test.shape)
print(y_train[0])
print(y_train.shape)
print(y_test[0])
print(y_test.shape)
print(torch.unique(y_train))

tensor([0.0545, 0.0558, 0.0565,  ..., 0.0463, 0.0463, 0.0463],
       dtype=torch.float64)
torch.Size([264, 8630])
tensor([0.0489, 0.0510, 0.0515,  ..., 0.0408, 0.0408, 0.0408],
       dtype=torch.float64)
torch.Size([36, 8630])
tensor(1)
torch.Size([264])
tensor(4)
torch.Size([36])
tensor([0, 1, 2, 3, 4, 5])
